In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import joblib
import tabulate as tb
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import Huber
from tensorflow.keras import Sequential, layers, optimizers, losses
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import os, random, numpy as np, tensorflow as tf
from model import FinancialLSTMModel

SEED = 42
os.environ["PYTHONHASHSEED"]=str(SEED)
os.environ["TF_DETERMINISTIC_OPS"]="1"
os.environ["TF_CUDNN_DETERMINISTIC"]="1"
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)

In [3]:
CSV_PATH = './../data/AAPL_1min.csv'
DATE_COL = 'Datetime'

SEQ_LENGTH = 60
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCHS = 100
TEST_RATIO = 0.2
VAL_SPLIT = 0.2

REPS = 3

EXCLUDE_COLUMNS = ['Datetime', 'returns', 'direction'] 
FEATURES = []

TARGET = 'direction'

def build_hidden_layers1():
    return [
        tf.keras.layers.LSTM(96, return_sequences=True, recurrent_dropout=0.1),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.LSTM(96, return_sequences=False, recurrent_dropout=0.1),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
    ]


def build_hidden_layers2():
    return [
        tf.keras.layers.LSTM(50, return_sequences=True),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.LSTM(50, return_sequences=False),
        tf.keras.layers.Dense(64, activation='relu'),
    ]
    
def build_hidden_layers3():
    return [
        tf.keras.layers.Conv1D(filters=64, kernel_size=2, activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.LSTM(50, return_sequences=True),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.LSTM(50, return_sequences=False),
        tf.keras.layers.Dense(64, activation='relu'),
    ]

In [4]:
if len(FEATURES) == 0:
    data = pd.read_csv(CSV_PATH, parse_dates=[DATE_COL])
    all_cols = data.columns.tolist()
    FEATURES = [(feat, "minmax") for feat in all_cols if feat not in EXCLUDE_COLUMNS]
    print("Using features:", FEATURES)

Using features: [('Open', 'minmax'), ('High', 'minmax'), ('Low', 'minmax'), ('Close', 'minmax'), ('% change', 'minmax'), ('rsi_14', 'minmax'), ('rsi_28', 'minmax'), ('rsi_50', 'minmax'), ('rsi_7', 'minmax'), ('macd', 'minmax'), ('ema_10', 'minmax'), ('ema_20', 'minmax'), ('ema_50', 'minmax'), ('ema_100', 'minmax'), ('ema_200', 'minmax'), ('stoch_k', 'minmax'), ('stoch_d', 'minmax'), ('roc', 'minmax'), ('adx', 'minmax'), ('di_plus', 'minmax'), ('di_minus', 'minmax'), ('atr_14', 'minmax'), ('atr_20', 'minmax'), ('close_pos', 'minmax'), ('body_range_ratio', 'minmax'), ('bb_lower_20', 'minmax'), ('bb_middle_20', 'minmax'), ('bb_upper_20', 'minmax'), ('bb_width_20', 'minmax')]


In [5]:
res = []

for r in range(REPS):
    print(f"--- REPETITION {r+1}/{REPS} ---")
    model = FinancialLSTMModel(
        csv_path=CSV_PATH,
        features_scales=FEATURES,
        target_col="direction",
        datetime_col="Datetime",

        seq_length=SEQ_LENGTH,
        batch_size=BATCH_SIZE,
        learning_rate=LEARNING_RATE,
        epochs=EPOCHS,
        test_ratio=TEST_RATIO,
        val_split=VAL_SPLIT,
    )

    model.prepare_data()
    model.build_model(build_hidden_layers3())
    model.train()
    ev = model.evaluate()
    res.append(ev)
    
    print(f">> results: {ev}")

--- REPETITION 1/3 ---


2025-11-29 17:56:59.468758: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-11-29 17:57:06.099900: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 1263072960 exceeds 10% of free system memory.


Epoch 1/100


2025-11-29 17:57:11.572173: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 1263072960 exceeds 10% of free system memory.
2025-11-29 17:57:14.032579: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


5671/5672 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5015 - loss: 0.6935

2025-11-29 18:00:07.707141: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 315761280 exceeds 10% of free system memory.
2025-11-29 18:00:07.941496: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 315761280 exceeds 10% of free system memory.
2025-11-29 18:00:08.206927: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatase

5672/5672 ━━━━━━━━━━━━━━━━━━━━ 188s 31ms/step - accuracy: 0.4996 - loss: 0.6933 - val_accuracy: 0.4852 - val_loss: 0.6933 - learning_rate: 0.0010
Epoch 2/100
4476/5672 ━━━━━━━━━━━━━━━━━━━━ 40s 34ms/step - accuracy: 0.4957 - loss: 0.6932

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(res)

print("\n=== SUMMARY ===")
print(tb.tabulate(df, headers='keys', tablefmt='pretty', showindex="always"))


=== SUMMARY ===
+---+--------------------------+---------------------+---------------------+--------------------+---------------------+--------------------+------------------------+----------------+---------------------+
|   | first_prediction_correct |      accuracy       |      f1_score       |     precision      |       recall        |      auc_roc       |    confusion_matrix    | last epoch num |  balanced_accuracy  |
+---+--------------------------+---------------------+---------------------+--------------------+---------------------+--------------------+------------------------+----------------+---------------------+
| 0 |          False           | 0.46889226100151743 |     0.31640625      | 0.5510203838348389 | 0.22191780805587769 | 0.5207762718200684 | [[228, 66], [284, 81]] |       23       | 0.49871400615040534 |
| 1 |          False           | 0.4673748103186646  |  0.162291169451074  | 0.6296296119689941 | 0.09315068274736404 | 0.5283617377281189 | [[274, 20], [331, 34]]